In [139]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [140]:
conda install -c conda-forge parse

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [141]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
print("Imported!")
import lxml.html as lh
import folium

Imported!


In [15]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [16]:
extract_data = requests.get(url).text
data = BeautifulSoup(extract_data, 'lxml')

In [19]:
cols = ['Postalcode','Borough','Neighborhood']
TO = pd.DataFrame(columns = cols)
content = data.find('div', class_='mw-parser-output')
table = content.table.tbody
Postalcode = 0
Borough = 0
Neighborhood = 0

In [34]:
# Parsing through the data

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            Postalcode = td.text
            i = i + 1
        elif i == 1:
            Borough = td.text
            i = i + 1
        elif i == 2: 
            Neighborhood = td.text.strip('\n').replace(']','')
    TO = TO.append({'Postalcode': Postalcode,'Borough': Borough,'Neighborhood': Neighborhood},ignore_index=True)

In [35]:
TO = TO[TO.Borough!='Not assigned']
TO = TO[TO.Borough!= 0]
TO.reset_index(drop = True, inplace = True)
i = 0


In [36]:
# Now to create the loop for 
for i in range(0,TO.shape[0]):
    if TO.iloc[i][2] == 'Not assigned':
        TO.iloc[i][2] = TO.iloc[i][1]
        i = i+1
        



In [37]:
df = TO.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [48]:
# Dropping Nan values and Not Assigned values from the dataframe
df = df.dropna()
df = df[~df.Borough.str.contains("Not assigned")]

In [49]:
def neighborhood(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_final = grp.apply(neighborhood).reset_index(name='Neighborhood')

In [50]:
print(df_final.shape)

(103, 3)


In [51]:
df_final

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae
...,...,...,...
98,M9N\n,York\n,Weston
99,M9P\n,Etobicoke\n,Westmount
100,M9R\n,Etobicoke\n,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."


In [52]:
df_final.to_csv('TO.csv', index=False)